In [66]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from freetextmethods import word_counts
from compressdf import compress_df

In [72]:
pd.options.mode.use_inf_as_na = True
DATA_FOLDER = '/home/roger/code/kaggle/predict-future-sales/data'
SUBMISSION_FOLDER = "comp_submissions/"
index_vars = ["date_block_num", "shop_id", "item_id"]

transactions       = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items              = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories    = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
item_categories_en = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories_en.csv'))
shops              = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
shops_en           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops_en.csv'))
test_ids           = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
sample             = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))

# Adding English translations to item_categories and shops
item_categories = item_categories.merge(
    item_categories_en[["item_category_id", "item_category_name"]]
        .rename(columns={"item_category_name":"item_category_name_en"}),
    on="item_category_id")
item_categories.head()

shops = shops.merge(
    shops_en[["shop_id", "shop_name"]]
        .rename(columns={"shop_name":"shop_name_en"}),
    on="shop_id")

In [121]:
# Aggregate by sale month + append test set
sales_by_month = (transactions[index_vars + ["item_cnt_day"]]
                   .groupby(index_vars)
                   .agg("sum")
                   .rename(columns={"item_cnt_day":"item_cnt_month"})
                   .reset_index())

sales_by_month["item_cnt_month_orig"] = sales_by_month["item_cnt_month"]
sales_by_month["item_cnt_month"] = sales_by_month["item_cnt_month"].clip(0, 20)

In [99]:
sales_by_month_skeleton = pd.DataFrame()

In [103]:
m = 1
shop_ids = sales_by_month[sales_by_month["date_block_num"] == m]["shop_id"].unique()
item_ids = sales_by_month[sales_by_month["date_block_num"] == m]["item_id"].unique()
#   items sold cross product shops with sales during month m
cross_join = pd.merge(left=pd.Series(shop_ids, name="shop_id"), 
                      right=pd.Series(item_ids, name="item_id"), 
                      how="cross")
cross_join["date_block_num"] = m
if m>0:
    cross_join.index = cross_join.index + sales_by_month_skeleton.index.max() + 1
sales_by_month_skeleton = pd.concat([sales_by_month_skeleton, cross_join], axis="rows")

In [ ]:
sales_by_month = sales_by_month.merge(
    cross_join,
    on=index_vars,
    how="outer")
sales_by_month["item_cnt_month"] = sales_by_month[["item_cnt_month", "item_cnt_month_zeros"]].max(axis="columns")

In [79]:
sales_by_month.head().T

,0,1,2,3,4
date_block_num,0.0,0.0,0.0,0.0,0.0
shop_id,0.0,0.0,0.0,0.0,0.0
item_id,32.0,33.0,35.0,43.0,51.0
item_cnt_month,6.0,3.0,1.0,1.0,2.0
item_cnt_month_orig,6.0,3.0,1.0,1.0,2.0
item_cnt_month_zeros,0.0,0.0,0.0,0.0,0.0


In [119]:
FINAL_MONTH = 33
sales_by_month_skeleton = pd.DataFrame()

for m in range(FINAL_MONTH + 1):
    shop_ids = sales_by_month[sales_by_month["date_block_num"] == m]["shop_id"].unique()
    item_ids = sales_by_month[sales_by_month["date_block_num"] == m]["item_id"].unique()
    #   items sold cross product shops with sales during month m
    cross_join = pd.merge(left=pd.Series(shop_ids, name="shop_id"), 
                          right=pd.Series(item_ids, name="item_id"), 
                          how="cross")
    cross_join["date_block_num"] = m
    if m>0:
        cross_join.index = cross_join.index + sales_by_month_skeleton.index.max() + 1
    sales_by_month_skeleton = pd.concat([sales_by_month_skeleton, cross_join], axis="rows")

sales_by_month_skeleton["item_cnt_month_zeros"] = 0.0

0 365175
1 375728
2 381892
3 374670
4 375255
5 389988
6 386630
7 361395
8 354420
9 369794
10 364005
11 389804
12 347346
13 328164
14 342816
15 331926
16 327565
17 327369
18 330550
19 324513
20 303550
21 329368
22 316100
23 330250
24 306950
25 284491
26 279772
27 257372
28 232452
29 224288
30 228889
31 214536
32 218655
33 238172


In [122]:
sales_by_month = sales_by_month.merge(
            sales_by_month_skeleton,
            on=index_vars,
            how="outer")
sales_by_month["item_cnt_month"] = sales_by_month[["item_cnt_month", "item_cnt_month_zeros"]].max(axis="rows")
sales_by_month = sales_by_month.drop(columns="item_cnt_month_zeros")